# Solomon's notebook - Turnover
## To visualize percentage share of most traded stock in HKEX for the last 5 trading days.

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import plotly.express as px

from lib import hkex
from lib import holidays

today = datetime.now()
yesterday = today - timedelta(days=1)
num_days = 5

try:
    bdates = pd.bdate_range(end=today, periods=num_days, holidays=holidays.hk_holidays, freq='C')[::-1]
    dfs = list(map(hkex.get_price, bdates))
except hkex.NoQuotationsException:
    bdates = pd.bdate_range(end=yesterday, periods=num_days, holidays=holidays.hk_holidays, freq='C')[::-1]
    dfs = list(map(hkex.get_price, bdates))

In [ ]:
topN = 20
code_industry_map = hkex.get_all_industry()

for d, df in zip(bdates, dfs):
    #dfn = df.nlargest(topN, ['turnover'])
    dfn = df
    dfn = pd.merge(dfn, code_industry_map, how="left", on=['code'])
    dfn['industry'] = dfn['industry'].fillna('Others')
    pie = px.sunburst(dfn, path=['industry', 'code'], values='turnover',
                           title=f"Turnover Share of the top {topN} as of date {d:%Y-%m-%d}")
    pie.show()